<a href="https://colab.research.google.com/github/Jenoprabakar/TensorFlow-lab_experiments/blob/main/data_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Write a TensorFlow program that creates a data pipeline using tf.data for the Wine Quality dataset (including normalization, batching, and shuffling), builds and trains a feedforward neural network for wine quality classification, saves the trained model in both SavedModel and HDF5 formats, and finally reloads the saved models to evaluate their accuracy on the test dataset.


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
data = pd.read_csv('winequality-red.csv') # replace with your path
X = data.drop('quality', axis=1).values
y = data['quality'].values
y_classes = np.where(y <= 5, 0, np.where(y == 6, 1, 2))
X_train, X_test, y_train, y_test = train_test_split(
X, y_classes, test_size=0.2, random_state=42
)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
batch_size = 32
2303717624321023


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(batch_size)
model = keras.Sequential([
layers.Input(shape=(X_train.shape[1],)),
layers.Dense(64, activation='relu'),
layers.Dense(32, activation='relu'),
layers.Dense(3, activation='softmax') # 3 classes
])
model.compile(
optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy']
)
model.summary()
history= model.fit(
train_dataset,
validation_data=test_dataset,
epochs=50
)
model.save('wine_model.keras')
model.save('wine_model.h5')
loaded_keras = tf.keras.models.load_model('wine_model.keras')
loss, acc = loaded_keras.evaluate(test_dataset)
print(f"Keras (.keras) model accuracy: {acc:.4f}")
loaded_h5 = tf.keras.models.load_model('wine_model.h5')
loss, acc = loaded_h5.evaluate(test_dataset)
print(f"HDF5 (.h5) model accuracy: {acc:.4f}")
